In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import os

In [6]:
os.chdir('C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI\\validation_weight')

In [42]:
class TextCNN2d(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(TextCNN2d, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=(1, 1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=(1, 1))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(128 * (input_size//2//2//2) * (849//2//2//2), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # 차원 추가: (batch_size, 1, seq_length, embedding_dim)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # 플래튼
        x = self.dropout(x)
        x = self.fc(x)
        return x

def load_data(file_path, feature_path):
    df = pd.read_csv(file_path)
    documents = df['cleaned_text_normalized'].tolist()
    labels = df['type'].apply(lambda x: 1 if 'P' in x else 0).tolist()
    features = np.load(feature_path)
    return documents, labels, features


def build_dataloader(X, y, batch_size):
    tensor_x = torch.tensor(X).float()  
    tensor_y = torch.tensor(y).long()
    dataset = TensorDataset(tensor_x, tensor_y) 
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    return dataloader


def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    #가능한 경우, CuDNN 백엔드의 결정론적 모드 활성화
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss, total_accuracy, all_predictions, all_targets = 0, 0, [], []
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, pred = torch.max(output, dim=1)
        all_predictions.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        
        accuracy = torch.sum(pred == target).item() / len(target)
        total_accuracy += accuracy
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    avg_f1 = f1_score(all_targets, all_predictions, average='binary')
    avg_precision = precision_score(all_targets, all_predictions, average='binary')
    avg_recall = recall_score(all_targets, all_predictions, average='binary')
    
    return avg_loss, avg_accuracy, avg_f1, avg_precision, avg_recall



def test(model, test_loader, criterion, device):
    model.eval()
    total_loss, total_accuracy, all_predictions, all_targets = 0, 0, [], []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
            _, pred = torch.max(output, dim=1)
            all_predictions.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            accuracy = torch.sum(pred == target).item() / len(target)
            total_accuracy += accuracy
    
    avg_loss = total_loss / len(test_loader)
    avg_accuracy = total_accuracy / len(test_loader)
    avg_f1 = f1_score(all_targets, all_predictions, average='binary')
    avg_precision = precision_score(all_targets, all_predictions, average='binary')
    avg_recall = recall_score(all_targets, all_predictions, average='binary')
    
    return avg_loss, avg_accuracy, avg_f1, avg_precision, avg_recall


def run_training(file_path, feature_path, batch_size, input_size, hidden_size, num_classes, lr, epochs, seed=42):
    # 시드 값 설정
    set_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device} device.")

    _, labels, features = load_data(file_path, feature_path)
    X_train_test, X_temp, y_train_test, y_temp = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)
    X_train, X_val, y_train, y_val = train_test_split(X_train_test, y_train_test, test_size=0.25, random_state=42, stratify=y_train_test)
    
    train_loader = build_dataloader(X_train, y_train, batch_size)
    val_loader = build_dataloader(X_val, y_val, batch_size)

    model = CNNClassifier(num_classes=num_classes, kernel_size=3, dropout_rate=0.5).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, eps=1e-8)
    
    # 조기 종료를 위한 설정
    best_accuracy = 0.0
    patience_counter = 0

    for epoch in range(epochs):
        train_loss, train_acc, train_f1, train_precision, train_recall = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, val_f1, val_precision, val_recall = test(model, val_loader, criterion, device)  # 재사용하여 검증 세트에서 성능 평가
        
         # 검증 세트에서의 정확도가 이전 최고보다 좋은지 확인
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            patience_counter = 0  # 성능 개선 시 카운터 초기화   
            # 모델 가중치 저장 (최고 성능 모델)
            torch.save(model.state_dict(), f"./{val_acc:.2f}_best_model.pth")
        else:
            patience_counter += 1  # 성능 개선 없을 시 카운터 증가

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} / Accuracy: {train_acc:.4f} / Precision: {train_precision:.4f} / Recall: {train_recall:.4f} / F1-score: {train_f1:.4f} \n Val Loss: {val_loss:.4f} / Val Accuracy: {val_acc:.4f} /Val Precision: {val_precision:.4f} / Val Recall: {val_recall:.4f} / Val F1-score: {val_f1:.4f}")
        
        # 조기 종료 조건 확인
        if patience_counter >= 5:
            print("Early stopping triggered. Stopping training...")
            break  # 학습 종료


file_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI\\MBTI_preprocessed2.2.csv"
feature_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI\\cls\\BERT\\BERT_concat_cls_MBTI_nospec.npy"
batch_size = 16
input_size = 768 + 81   # Assuming BERT [CLS] token output size + LIWC feature size
hidden_size = 849  # Example hidden size, adjust as needed
num_classes = 2
lr = 2e-4
epochs = 20

In [43]:
run_training(file_path, feature_path, batch_size, input_size, hidden_size, num_classes, lr, epochs, seed=42)

Using cpu device.


KeyboardInterrupt: 

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 시드 고정
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 가정: 테스트 데이터셋 경로 설정
test_file_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI\\MBTI_preprocessed2.2.csv"
test_feature_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI\\cls\\BERT\\BERT_concat_cls_MBTI_nospec.npy"

# 모델 초기화 및 가중치 불러오기
model_path = "0.61_best_model.pth"  # 이전에 저장된 최고 성능 모델의 경로
model = CNNClassifier(num_classes=num_classes, kernel_size=3, dropout_rate=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.to(device)

# 모델을 평가 모드로 설정
model.eval()

# 테스트 데이터 로딩 및 DataLoader 설정
_, test_labels, test_features = load_data(test_file_path, test_feature_path)
test_loader = build_dataloader(test_features, test_labels, batch_size)

# 모델 평가
def evaluate(model, data_loader, device):
    total_loss, total_accuracy, all_predictions, all_targets = 0, 0, [], []
    with torch.inference_mode():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            _, pred = torch.max(output, dim=1)
            all_predictions.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            accuracy = (pred == target).sum().item() / len(target)
            total_accuracy += accuracy
    
    avg_accuracy = total_accuracy / len(data_loader)
    avg_f1 = f1_score(all_targets, all_predictions, average='binary')
    avg_precision = precision_score(all_targets, all_predictions, average='binary')
    avg_recall = recall_score(all_targets, all_predictions, average='binary')
    
    return avg_accuracy, avg_f1, avg_precision, avg_recall

# 평가 실행
avg_accuracy, avg_f1, avg_precision, avg_recall = evaluate(model, test_loader, device)
print(f"Test Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")

Test Accuracy: 0.65, Precision: 0.64, Recall: 0.95, F1-Score: 0.77
